# **Product Price Prediction**

### PriceSense AI is a Large Language Model (LLM)–based pricing system that predicts the market price of consumer products directly from their textual descriptions.



In [1]:
!pip install -q --upgrade bitsandbytes trl
!wget -q https://raw.githubusercontent.com/ed-donner/llm_engineering/main/week7/util.py -O util.py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.9/532.9 kB 27.4 MB/s eta 0:00:00


# **Importing libraries**

In [2]:
# imports

import torch
from functools import partial

from huggingface_hub import login
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    set_seed
)
from peft import PeftModel, PeftConfig
from datasets import load_dataset
from util import evaluate


login()


# **Defining Configurations**

In [3]:
# Constants

# Base model
BASE_MODEL = "Qwen/Qwen2.5-3B"

# Project info
PROJECT_NAME = "price"
HF_USER = "Adityaj18"

# Dataset (public, working)
LITE_MODE = False
DATA_USER = "ed-donner"
DATASET_NAME = (
    f"{DATA_USER}/items_prompts_lite"
    if LITE_MODE
    else f"{DATA_USER}/items_prompts_full"
)

HUB_MODEL_NAME = f"{HF_USER}/price-2025-11-28_18.47.07"

# Quantization
QUANT_4_BIT = True
capability = torch.cuda.get_device_capability()
use_bf16 = capability[0] >= 8


# **Quantization Configuration**

In [4]:

if QUANT_4_BIT:
    quant_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.bfloat16 if use_bf16 else torch.float16,
    )
else:
    quant_config = BitsAndBytesConfig(
        load_in_8bit=True,
        bnb_8bit_compute_dtype=torch.bfloat16 if use_bf16 else torch.float16,
    )


# **Loading Tokenizer and Base Model**

In [5]:
# Tokenizer and the Model

tokenizer = AutoTokenizer.from_pretrained(
    BASE_MODEL,
    trust_remote_code=True
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

tokenizer.padding_side = "right"

# Base model
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=quant_config,
    device_map="auto",
    trust_remote_code=True
)

base_model.generation_config.pad_token_id = tokenizer.pad_token_id


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

# **Trying Loading LoRA Adapter**

In [6]:

fine_tuned_model = base_model

try:
    peft_config = PeftConfig.from_pretrained(HUB_MODEL_NAME)
    fine_tuned_model = PeftModel.from_pretrained(
        base_model,
        HUB_MODEL_NAME
    )
    print("✅ Loaded LoRA adapter")

except Exception as e:
    print("⚠️ LoRA adapter not found. Using base model.")
    print("Reason:", e)

print(f"Memory footprint: {fine_tuned_model.get_memory_footprint() / 1e6:.1f} MB")


⚠️ LoRA adapter not found. Using base model.
Reason: Can't find 'adapter_config.json' at 'Adityaj18/price-2025-11-28_18.47.07'
Memory footprint: 2010.1 MB


# **Loading Dataset**

In [7]:
dataset = load_dataset(DATASET_NAME)
test = dataset["test"]
print("Test samples:", len(test))


README.md:   0%|          | 0.00/520 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/172M [00:00<?, ?B/s]

data/val-00000-of-00001.parquet:   0%|          | 0.00/2.15M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/2.18M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/800000 [00:00<?, ? examples/s]

Generating val split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Test samples: 10000


# **Creating a Prediction Function**

In [8]:
def model_predict(item, model):
    inputs = tokenizer(
        item["prompt"],
        return_tensors="pt"
    ).to(next(model.parameters()).device)

    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=8,
            do_sample=False,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id
        )

    prompt_len = inputs["input_ids"].shape[1]
    generated_ids = output_ids[0, prompt_len:]
    return tokenizer.decode(generated_ids, skip_special_tokens=True)


In [9]:
predict_fn = partial(model_predict, model=fine_tuned_model)


# **Evaluation**

In [11]:
def predict_wrapper(item):
    return model_predict(item, fine_tuned_model)

set_seed(42)
evaluate(predict_wrapper, test)
